# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
# from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

# from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.37.0


In [2]:
ws = Workspace.from_config()
# print(ws.datasets) #see all availabel worksapce datasets
# my_dataset=Dataset.get_by_name(workspace=ws, name="our_name") #get datasets by name - given in the previous command
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'palsy'
# project_folder = './palsy-project'


experiment=Experiment(ws, experiment_name)
print(experiment)

quick-starts-ws-178503
aml-quickstarts-178503
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21
Experiment(Name: palsy,
Workspace: quick-starts-ws-178503)


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name) 
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:


# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bellpalsy"
description_text = "Datapoints of face"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
#         datastore = Datastore.get(ws, datastore_name='palda')
#         example_data=datastore
        path="https://raw.githubusercontent.com/nsourlos/palda_azure/main/palda.csv"
#         example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(path=path)#path = [(datastore, 'palda.csv')])#example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
df=dataset.to_pandas_dataframe()

df = df.iloc[: , :-1]

# df = dataset.to_pandas_dataframe()
df.describe()
print(df)

     x_1  y_1  x_2  y_2  x_3  y_3  x_4  y_4  x_5  y_5  ...  y_64  x_65  y_65  \
0    132  439  115  496  111  566  134  633  177  688  ...   581   588   583   
1    194  402  186  444  182  486  199  543  230  591  ...   490   542   507   
2     76  379   78  455   86  559  121  631  161  704  ...   628   654   629   
3    201  366  203  369  217  415  230  444  249  467  ...   418   487   419   
4    153  519  157  563  144  614  151  684  166  757  ...   672   520   686   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
198  227  226  234  257  244  287  254  315  268  340  ...   303   402   303   
199  131  168  128  184  125  200  123  217  125  233  ...   219   214   226   
200  132  231  134  254  136  278  139  303  148  327  ...   312   269   312   
201   62  103   63  116   64  127   66  139   69  151  ...   138   126   141   
202   -2  378    5  465   20  556   37  650   63  745  ...   660   531   673   

     x_66  y_66  x_67  y_67  x_68  y_68

In [5]:
# for i in range(len(df)):
#     if df['diagnosis'][i]=='peripheral':
#         df['diagnosis'][i]=0
#     elif df['diagnosis'][i]=='central':
#         df['diagnosis'][i]=1
#     elif df['diagnosis'][i]=='Healthy':
#         df['diagnosis'][i]=3
# np.unique(df['diagnosis'])
# df['diagnosis']

In [6]:
# df

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
       "--C": choice (0.01,0.1,1),
       "--max_iter": choice(4,8,16,32,64,128,256) #choice(10,20,40,60,100,150,200)
   })

if "training" not in os.listdir():
    os.mkdir("./training")

#Copied from https://knowledge.udacity.com/questions/386233 so that we won't get an error with widget loading
#Also model save commands were added to train.py file from the same link
# script_folder='./training'
# os.makedirs(script_folder,exist_ok=True)
# import shutil
# shutil.copy('./train.py',script_folder)
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn( source_directory='.',compute_target=compute_cluster,entry_script="train.py")
#script_folder
hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,max_concurrent_runs=4,policy=early_termination_policy, estimator=estimator)
#max_concurrent_runs=5

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [8]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails

hyperdrive_run=experiment.submit(config=hyperdrive_run_config) 
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6631d5fc-0506-411f-b8ba-5d921207f065
Web View: https://ml.azure.com/runs/HD_6631d5fc-0506-411f-b8ba-5d921207f065?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-178503/workspaces/quick-starts-ws-178503&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-25T11:07:21.591689][API][INFO]Experiment created<END>\n""<START>[2022-01-25T11:07:22.641280][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-25T11:07:23.316965][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_6631d5fc-0506-411f-b8ba-5d921207f065
Web View: https://ml.azure.com/runs/HD_6631d5fc-0506-411f-b8ba-5d921207f065?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-178503/workspaces/quick-starts-ws-178503&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_6631d5fc-0506-411f-b8ba-5d921207f065',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-25T11:07:21.312391Z',
 'endTimeUtc': '2022-01-25T11:16:27.604812Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '71baa99e-8686-41e4-a36c-c7bfffa3d448',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '21',
  'score': '0.8780487804878049',
  'best_child_run_id': 'HD_6631d5fc-0506-411f-b8ba-5d921207f065_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg178503.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_6631d5fc-0506-411f-b8ba-5d921207f065/azureml-lo

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
# from azureml.widgets import RunDetails
# RunDetails(hyperdrive_run).show()
# hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [19]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
# if an error occurs it suggests that the best_run has the value equivalence of null in other languages, which means 
#it does not contain anything. This means that the there was no result from hyperdrive_run.get_best_run_by_primary_metric() 
#as used in the notebook to retrieve the best run, and thus nothing was assigned the best_run object. 
#The question then would be: did the training complete successfully?

print("best run metrics :", best_run.get_metrics())
#we can also use 'hyperdrive_best_run.get_details()' and 'hyperdrive_best_run.get_file_names()'
# if "outputs" not in os.listdir():
#     os.mkdir("./outputs")
# os.makedirs('outputs')
# joblib.dump('outputs/hyper_best')
model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')
#Taken from https://github.com/Azure/MachineLearningNotebooks/issues/269

Run(Experiment: palsy,
Id: HD_6631d5fc-0506-411f-b8ba-5d921207f065_0,
Type: azureml.scriptrun,
Status: Completed)
best run metrics : {'Regularization Strength:': 0.01, 'Max iterations:': 128, 'Accuracy': 0.8780487804878049}


TypeError: dump() missing 1 required positional argument: 'filename'

In [14]:
#TODO: Save the best model
# hyperdrive_run.get_metrics()

# import onnxmltools

# best_run2, fitted_model = hyperdrive_run.get_output(return_onnx_model=True)
# # Taken from https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

# onnxmltools.utils.save_model(fitted_model, '/outputs/best_hyper_drive.onnx')

AttributeError: 'HyperDriveRun' object has no attribute 'get_output'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model=best_run.register_model(model_name='hyperdrive_model',model_path='./outputs/model.joblib')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

